<a href="https://colab.research.google.com/github/NateMophi/SCC-454/blob/main/LAB2/SCC454_Lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install pyspark==4.0.0 -q

# Java Installation
!apt-get install openjdk-11-jdk-headless -qq > /dev/null

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.1/434.1 MB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [3]:
# Set Java environmenr variable
import os
os.environ["JAVA HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"

print("PySpark & Java installed successfully!")

PySpark & Java installed successfully!


In [4]:
from pyspark.sql import SparkSession
from pyspark import SparkConf

# Session Creation
spark = SparkSession.builder.appName("SCC454-SparkIntro").config("spark.driver.memory", "4g")\
        .config("spark.ui.port", "4050")\
        .getOrCreate()

  # Underlying Context
sc = spark.sparkContext
print(f"Spark Version: {spark.version}")
print(f"Spark App Name: {spark.sparkContext.appName}")
print(f"Master: {spark.sparkContext.master}")

Spark Version: 4.0.0
Spark App Name: SCC454-SparkIntro
Master: local[*]


# **Resilient Distributed Datasets (RDDs)**

In [5]:
# RDD Creation

# 1a) Parallelization from a Python Collection
nums = [1,2,3,4,5,6,7,8,9,10]
nums_rdd =sc.parallelize(nums)

type(nums_rdd)
print(f"Number of paritions: {nums_rdd.getNumPartitions()}")
print(f"First 5 elements : {nums_rdd.take(5)}")



Number of paritions: 2
First 5 elements : [1, 2, 3, 4, 5]


In [6]:
# 1b) Parallelize with a specific number of partitions
nums_rdd_4part = sc.parallelize(nums, 4)
print(f"Number of partitions: {nums_rdd_4part.getNumPartitions()}")

# View Partitions
print("\nData in each partition: ")
print(nums_rdd_4part.glom().collect())

Number of partitions: 4

Data in each partition: 
[[1, 2], [3, 4], [5, 6], [7, 8, 9, 10]]


In [7]:
# 2) Create RDD from text file
sample_text = """Apache Spark is a unified analytics engine for large-scale data processing.
It provides high-level APIs in Java, Scala, Python and R.
Spark powers a stack of libraries including SQL and DataFrames.
It also includes MLlib for machine learning and GraphX for graph processing.
Spark can run on Hadoop, Apache Mesos, Kubernetes, standalone, or in the cloud."""


with open("spark_intro.txt", "w") as f:
  f.write(sample_text)

# Load text file as RDD
text_rdd = sc.textFile("spark_intro.txt")
print(f"Number of lines: {text_rdd.count()}")
print("\n1st 3 lines:")
for line in text_rdd.take(3):
  print(f" - {line}")

Number of lines: 5

1st 3 lines:
 - Apache Spark is a unified analytics engine for large-scale data processing.
 - It provides high-level APIs in Java, Scala, Python and R.
 - Spark powers a stack of libraries including SQL and DataFrames.


# **TRANSFORMATIONS** & **ACTIONS**

In [9]:
numbers = sc.parallelize(range(1,11))

# map: Apply function to each element
squared = numbers.map(lambda x: x**2)
print(f"Original: {numbers.collect()} ")
print(f"Squared: {squared.collect()} ")

# filter: Keep Only elements that match a condition
evens = numbers.filter(lambda x: x%2==0)
print(f"Evens: {evens.collect()} ")



Original: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10] 
Squared: [1, 4, 9, 16, 25, 36, 49, 64, 81, 100] 
Evens: [2, 4, 6, 8, 10] 
